In [40]:
import pandas as pd
from tsai.all import *
import numpy as np

from sklearn.model_selection import train_test_split

from mlxtend.evaluate.time_series import (
    GroupTimeSeriesSplit,
    plot_splits,
    print_cv_info,
    print_split_info,
)

from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [31]:
from importlib.machinery import SourceFileLoader
SourceFileLoader(
    'tcn', r'C:\Users\gijst\Documents\Uni\scriptie\RUL-prediction\keras-tcn-master\tcn\tcn.py').load_module()

<module 'tcn' from 'C:\\Users\\gijst\\Documents\\Uni\\scriptie\\RUL-prediction\\keras-tcn-master\\tcn\\tcn.py'>

In [2]:
def norm_df(df,cols):
  for col in cols:
    df[col] = df[col] / df[col].max()
  return df

In [3]:
df_shift7 = pd.read_pickle('merge_df_shift_7.pickle')
df_shift7 = df_shift7[['date', 'NDVI_mean', 'NDMI_mean', 'NBSI_mean', 'name', 'tavg',  'prcp',
       'NDVI_mean_shift', 'tavg_shift', 'prcp_shift']]
df_shift14 = pd.read_pickle('merge_df_shift_14.pickle')
df_shift14 = df_shift14[['date', 'NDVI_mean', 'NDMI_mean', 'NBSI_mean', 'name', 'tavg',
        'prcp','NDVI_mean_shift', 'tavg_shift', 'prcp_shift']]

In [4]:

df_shift7 = norm_df(df_shift7, ['tavg','prcp','tavg_shift','prcp_shift'])
df_shift14 = norm_df(df_shift14, ['tavg','prcp','tavg_shift','prcp_shift'])
df_shift7.dropna(axis = 0, inplace = True)
df_shift14.dropna(axis = 0, inplace = True)

In [5]:
df_shift14.columns

Index(['date', 'NDVI_mean', 'NDMI_mean', 'NBSI_mean', 'name', 'tavg', 'prcp',
       'NDVI_mean_shift', 'tavg_shift', 'prcp_shift'],
      dtype='object')

In [6]:
x_cols = ['NDVI_mean', 'NDMI_mean', 'NBSI_mean', 'name', 'tavg_shift', 'prcp_shift']

In [7]:
sliding_window_X, sliding_window_y = SlidingWindowPanel(window_len=14, unique_id_cols=[
                                                        'name'], get_x=x_cols, get_y=['NDVI_mean_shift'], stride=1)(df_shift14)


processing data...


...data processed
concatenating X...
...X concatenated
concatenating y...
...y concatenated


In [35]:
df_shift14

,date,NDVI_mean,NDMI_mean,NBSI_mean,name,tavg,prcp,NDVI_mean_shift,tavg_shift,prcp_shift
0,2017-01-13,0.028000,-0.325800,0.004000,almelo,0.045307,0.229358,0.344275,0.022654,0.000000
1,2017-01-14,0.052914,-0.308897,0.011951,almelo,0.048544,0.055046,0.363848,0.187702,0.000000
2,2017-01-15,0.077509,-0.292253,0.019659,almelo,-0.045307,0.010321,0.382967,0.152104,0.021789
3,2017-01-16,0.101775,-0.275888,0.027123,almelo,-0.100324,0.000000,0.401623,0.114825,0.108945
4,2017-01-17,0.125704,-0.259820,0.034345,almelo,-0.132686,0.000000,0.419806,0.077670,0.000000
...,...,...,...,...,...,...,...,...,...,...
49223,2022-12-08,0.891592,-0.446221,0.120424,woerden,0.074434,0.053899,0.791886,0.262136,0.047018
49224,2022-12-09,0.891439,-0.449849,0.118250,woerden,-0.045307,0.000000,0.772311,0.294498,0.134174
49225,2022-12-10,0.890687,-0.452987,0.116155,woerden,-0.012945,0.000000,0.751186,0.275081,0.001147
49226,2022-12-11,0.889263,-0.455576,0.114149,woerden,-0.006472,0.000000,0.728640,0.262136,0.048165


In [46]:
swp_X_T = np.moveaxis(sliding_window_X, 0, 1)
swp_X_T = np.moveaxis(swp_X_T, 0, -1)
swp_X_T = np.moveaxis(swp_X_T, 0, 1)
swp_X_T.shape

(14, 48906, 6)

In [39]:
sliding_window_X.shape

(48906, 6, 14)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(sliding_window_X, sliding_window_y,
                                                    random_state=42, shuffle=False, test_size=0.2)

In [15]:
model = TCN(c_in=6, c_out=1) 

In [29]:
import tensorflow as tf
from tcn import TCN, compiled_tcn
tf.config.list_physical_devices()


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [33]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
model = compiled_tcn(num_feat=swp_X_T.shape[-1], )

In [23]:
def compile_model(swp_X):

    # output tcn2
    nb_features = swp_X.shape[2]

    i = Input(shape=(swp_X.shape[1], nb_features))

    m = TCN(nb_filters=64, kernel_size=3, dropout_rate=0.6, dilations=[
            1, 2, 4, 8, 16, 32], return_sequences=True, activation='relu')(i)
    m = TCN(nb_filters=32, dropout_rate=0.6, kernel_size=3, dilations=[
            1, 2, 4, 8, 16, 32], return_sequences=False, activation='relu')(m)
    m = Dense(1, activation='linear')(m)
    model = Model(inputs=[i], outputs=[m])

    model.summary()
    opt = tf.keras.optimizers.Adam()
    model.compile(loss=root_mean_squared_error, optimizer=opt,
                  metrics=['mae', root_mean_squared_error])
    return model

array([0.5734838 , 0.58582443, 0.597576  , ..., 0.74152637, 0.7494    ,
       0.7586865 ], dtype=float32)

In [20]:
my_setup()

os              : Windows-10-10.0.22621-SP0
python          : 3.9.13
tsai            : 0.3.4
fastai          : 2.7.10
fastcore        : 1.5.27
torch           : 1.13.1+cpu
device          : cpu
cpu cores       : 6
threads per cpu : 2
RAM             : 15.76 GB
GPU memory      : [4.0] GB


In [21]:
check_data(sliding_window_X, sliding_window_y, splits=)

X      - shape: [48906 samples x 6 features x 14 timesteps]  type: ndarray  dtype:object  isnan: could not be checked
y      - shape: (48906,)  type: ndarray  dtype:float32  isnan: 0
